In [1]:
import tensorflow as tf
print(tf.__version__)  # TensorFlow 버전 확인
print("CUDA Available: ", tf.test.is_built_with_cuda())  # CUDA 사용 가능 여부 확인
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2024-12-25 10:38:16.138813: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-25 10:38:16.138889: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-25 10:38:16.138943: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2.14.0
CUDA Available:  True
Num GPUs Available:  1


In [2]:
import numpy as np
import cv2
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from mtcnn import MTCNN
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [3]:
# 테스트 데이터 로드
df_test = pd.read_csv('/workspace/#Ub370#Uc774#Ud130/test/label/test.csv')

# 이미지 경로 열 생성
df_test['img_path'] = df_test['path'] + df_test['filename']

In [4]:
# 데이터 업로드 제너레이터 설정
target_size = (224,224)     # 모델 입력에 맞는 사이즈 설정

# ImageDataGenerator 정의 
datagen = ImageDataGenerator() 

# test 데이터 generator 정의
test_generator = datagen.flow_from_dataframe(
    dataframe=df_test,                # Pandas DataFrame with image paths and labels
    directory=None,                   # Root directory for relative image paths
    x_col='img_path',                 # Column name in DataFrame containing image file paths
    y_col='faceExp_uploader',         # Column name in DataFrame containing labels
    target_size=target_size,           # Resize all images to (224, 224) pixels
    batch_size=32,                    # Number of images per batch
    class_mode='categorical'          # Multi-class classification: output one-hot encoded labels
)

print("Test Generator Class Indices:", test_generator.class_indices)


Found 1200 validated image filenames belonging to 4 classes.
Test Generator Class Indices: {'기쁨': 0, '당황': 1, '분노': 2, '슬픔': 3}


In [5]:
# 바운딩 박스 내부 이미지를 크롭하고 모델에 입력
def crop_and_predict(image, detections, emotion_classes, emotion_model):
    predictions = []
    for detection in detections:
        x, y, width, height = detection['box']
        x, y = max(x, 0), max(y, 0)  # 음수 좌표 방지
        face = image[y:y+height, x:x+width]  # 얼굴 영역 크롭
        face = tf.image.resize(face, (224, 224))  # 모델 입력 크기로 조정
        face = img_to_array(face) / 255.0  # 정규화
        face = np.expand_dims(face, axis=0)  # 배치 차원 추가
        
        # 감정 예측
        prediction = emotion_model.predict(face)
        emotion_index = np.argmax(prediction)
        emotion_label = emotion_classes[emotion_index]
        predictions.append((detection['box'], emotion_label, prediction[0][emotion_index]))

    return predictions

# MTCNN 및 감정 분류 모델 예측 결과와 실제 레이블 비교 함수
def evaluate_predictions(generator, detector, emotion_classes, emotion_model):
    y_true = []  # 실제 레이블
    y_pred = []  # 예측 레이블
    class_indices = generator.class_indices
    index_to_label = {v: k for k, v in class_indices.items()}  # 인덱스를 레이블로 변환

    for batch_index in range(len(generator)):
        images, labels = generator[batch_index]  # 배치 단위로 이미지와 레이블 가져오기
        
        for i, (image, label) in enumerate(zip(images, labels)):

            # MTCNN으로 얼굴 탐지
            detections = detector.detect_faces(image)
            if not detections:
                print(f"Image {batch_index * generator.batch_size + i}: No face detected.")
                continue
            
            # 바운딩 박스 내부 얼굴 감정 예측
            predictions = crop_and_predict(image, detections, emotion_classes, emotion_model)

            # 다중 얼굴 탐지 중 첫 번째 얼굴 기준으로 비교 (여러 얼굴 처리 필요 시 수정)
            if predictions:
                _, predicted_emotion, _ = predictions[0]  # 첫 번째 얼굴의 예측값 가져오기
                y_pred.append(predicted_emotion)
                true_label_index = np.argmax(label)  # 실제 레이블 (원핫 인코딩)
                true_label = index_to_label[true_label_index]
                y_true.append(true_label)
            
                print(f"Image {batch_index * generator.batch_size + i}:")
                print(f"  True: {true_label}, Predicted: {predicted_emotion}")
            else:
                print(f"Image {batch_index * generator.batch_size + i}: No emotion predicted.")
    
    # 결과 출력
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=emotion_classes))

    print("\nConfusion Matrix:")
    cm = confusion_matrix(y_true, y_pred, labels=emotion_classes)
    print(cm)




In [6]:

# Load MTCNN for face detection
detector = MTCNN()

# Load the pre-trained MobileNet model
mobilenet_model = tf.keras.models.load_model('/workspace/uk/Emotion_Classifier_MoblieNet.h5')

# 실행
evaluate_predictions(test_generator, detector, ['기쁨', '당황', '분노', '슬픔'], mobilenet_model)

2024-12-25 10:38:34.349239: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14297 MB memory:  -> device: 0, name: NVIDIA RTX A4000, pci bus id: 0000:04:00.0, compute capability: 8.6
2024-12-25 10:38:41.785852: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700


1/1 [==============================] - 1s 1s/step
Image 0:
  True: 슬픔, Predicted: 분노
1/1 [==============================] - 0s 33ms/step
Image 1:
  True: 당황, Predicted: 분노
1/1 [==============================] - 0s 32ms/step
Image 2:
  True: 슬픔, Predicted: 기쁨
1/1 [==============================] - 0s 32ms/step
Image 3:
  True: 당황, Predicted: 기쁨
1/1 [==============================] - 0s 32ms/step
Image 4:
  True: 분노, Predicted: 기쁨
1/1 [==============================] - 0s 31ms/step
Image 5:
  True: 당황, Predicted: 기쁨
1/1 [==============================] - 0s 31ms/step
Image 6:
  True: 당황, Predicted: 기쁨
1/1 [==============================] - 0s 31ms/step
Image 7:
  True: 슬픔, Predicted: 기쁨
1/1 [==============================] - 0s 31ms/step
Image 8:
  True: 기쁨, Predicted: 기쁨
1/1 [==============================] - 0s 33ms/step
Image 9:
  True: 당황, Predicted: 분노
Image 10: No face detected.
1/1 [==============================] - 0s 30ms/step
Image 11:
  True: 슬픔, Predicted: 기쁨
1/1 [===========